In [7]:
!pip install lxml # pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 228.4 kB/s eta 0:00:0000:0100:01


In [18]:
from bs4 import BeautifulSoup
import requests
import lxml


# define headers argument
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}

In [34]:
# query

search = "Attention is all you need"

# format the google scholar article link
# 'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=web+scraping&btnG='
base = 'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q='
end = '&btnG='
url = base + '+'.join(search.split()) + end

response=requests.get(url,headers=headers)

# soup query
soup=BeautifulSoup(response.content, "html.parser") # 'lxml')

In [35]:
#print(soup)



for item in soup.select('[data-lid]'):
	try:
		print('----------------------------------------')

		# title
		print(item.select('h3')[0].get_text())
		print("\n")

		# URL download link for the PDF
		print(item.select('a')[0]['href'])
		
		print(item.select('.gs_rs')[0].get_text())

		print('----------------------------------------')
		
	except Exception as e:
		#raise e
		print('')

# parse the most relevant articles (i.e. top 5?)

----------------------------------------
Attention is all you need


https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf
… to attend to all positions in the decoder up to and including that position. We need to prevent 
… We implement this inside of scaled dot-product attention by masking out (setting to −∞) …
----------------------------------------
----------------------------------------
Attention is all you need in speech separation


https://arxiv.org/pdf/2010.13154
… Transformers are emerging as a natural alternative to standard RNNs, replacing recurrent 
computations with a multi-head attention mechanism.In this paper, we propose the SepFormer…
----------------------------------------
----------------------------------------
Attention is all you need


http://www.aiotlab.org/teaching/intro2ai/slides/10_attention_n_bert.pdf
… Attention • Visual attention and textual attention … Could we build a transformer-based 
model whose language model lo